In [1]:
# tests/test_order_with_city_viewer.py
import sys, os, random
sys.path.insert(0, r"D:\LLMDelivery-LJ\SimWorld")
sys.path.insert(0, r"D:\LLMDelivery-LJ\LLM-Delivery")

from PyQt5.QtWidgets import QApplication
from simworld.utils.vector import Vector
from simworld.map.map import Map
from Base.Order import Order  # 你已在 Order 内实现 compute_with_map
from simworld.utils.city_viewer import CityViewer, build_city_map, thin_by_min_spacing_xy

CONFIG = {
    "map.input_roads": r"D:\LLMDelivery-LJ\Test_Data\test\roads.json",
    "traffic.sidewalk_offset": 100.0,
}

WORLD_JSON = r"D:\LLMDelivery-LJ\Test_Data\test\progen_world_enriched.json"
BUILDING_DEFS = r"D:\LLMDelivery-LJ\Test_Data\test\buildings.json"

def make_random_order_and_draw(viewer: CityViewer, city_map: Map):
    bxy = viewer.get_buildings_xy()
    if len(bxy) < 2:
        viewer.set_info_text("⚠️ 建筑数量不足。")
        return
    pick = random.choice(bxy)
    drop = random.choice(bxy)
    while drop == pick:
        drop = random.choice(bxy)

    items = random.sample(
        ["Burger","Fries","Pizza","Salad","Noodles","Coffee","Tea","Sushi","Taco"], 
        k=random.randint(1, 3)
    )
    note = random.choice(["", "Leave at door", "Call on arrival", "No contact delivery", "Extra napkins"])

    order = Order(
        pickup_address=Vector(pick[0], pick[1]),
        delivery_address=Vector(drop[0], drop[1]),
        items=items,
        special_note=note
    )
    path_nodes = order.compute_with_map(city_map)
    route_xy = [(n.position.x, n.position.y) for n in path_nodes]
    route_xy = thin_by_min_spacing_xy(route_xy, 1200.0)

    # 叠加到画布
    viewer.clear_overlays()
    viewer.plot_route(route_xy, color="#FF5722", width=4.0, scatter=False, show_endpoints=True)

    # 信息面板
    dist_m = order.distance_cm / 100.0
    eta_m, eta_s = int(order.eta_s // 60), int(order.eta_s % 60)
    tlim_m, tlim_s = int(order.time_limit_s // 60), int(order.time_limit_s % 60)
    viewer.set_info_text(
        "\n".join([
            f"Order ID: {order.id}",
            f"Pickup:  ({int(order.pickup_address.x)}, {int(order.pickup_address.y)})",
            f"Dropoff: ({int(order.delivery_address.x)}, {int(order.delivery_address.y)})",
            f"Items: {', '.join(order.items)}",
            f"Note: {order.special_note or '(none)'}",
            f"Distance: {dist_m:.1f} m",
            f"ETA: ~{eta_m}m {eta_s}s",
            f"Earnings: ${order.earnings:.2f}",
            f"Time Limit: {tlim_m}m {tlim_s}s",
        ])
    )

def main():
    app = QApplication(sys.argv)

    # 共享底图 + 共享 Map（后续各种 test 都能直接使用）
    viewer = CityViewer(WORLD_JSON, BUILDING_DEFS, title="Order Test (Reusable City Viewer)")
    city_map = build_city_map(CONFIG)

    # 加一个按钮：一键生成新订单并可视化
    viewer.add_button("New Random Order", lambda: make_random_order_and_draw(viewer, city_map))
    viewer.add_button("Save Image", lambda: viewer.save_png(os.path.join(os.path.dirname(WORLD_JSON), "order_test_view.png")))

    # 先出一单
    make_random_order_and_draw(viewer, city_map)

    viewer.show()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SystemExit: 0

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
